# 🕸️ NEXUS AI - Network Graph Analysis\n\n**Objective:** Analyze transaction networks to identify money laundering rings and complex schemes.\n\n**Contents:**\n1. Network Construction\n2. Community Detection\n3. Centrality Analysis\n4. Suspicious Pattern Detection\n5. Visualization

In [ ]:
# Importsimport pandas as pdimport numpy as npimport matplotlib.pyplot as pltimport networkx as nxfrom collections import Counterimport warningswarnings.filterwarnings('ignore')print("✅ Network analysis libraries loaded")

## 1️⃣ Network Construction\n\nBuild transaction network from sender-receiver relationships.

In [ ]:
# Generate synthetic transaction networknp.random.seed(42)n_nodes = 100n_transactions = 500# Create transaction edgesedges = []amounts = []for _ in range(n_transactions):    sender = f"CUST-{np.random.randint(1, n_nodes):03d}"    receiver = f"CUST-{np.random.randint(1, n_nodes):03d}"    if sender != receiver:        edges.append((sender, receiver))        amounts.append(np.random.lognormal(9, 1))# Create graphG = nx.DiGraph()for (sender, receiver), amount in zip(edges, amounts):    if G.has_edge(sender, receiver):        G[sender][receiver]['weight'] += amount        G[sender][receiver]['count'] += 1    else:        G.add_edge(sender, receiver, weight=amount, count=1)print(f"🕸️ Network created:")print(f"   Nodes: {G.number_of_nodes()}")print(f"   Edges: {G.number_of_edges()}")print(f"   Density: {nx.density(G):.4f}")print(f"   Connected: {nx.is_weakly_connected(G)}")

## 2️⃣ Community Detection\n\nIdentify clusters of closely connected accounts (potential layering schemes).

In [ ]:
from networkx.algorithms import community# Convert to undirected for community detectionG_undirected = G.to_undirected()# Louvain community detectioncommunities = community.louvain_communities(G_undirected, seed=42)print(f"👥 Communities detected: {len(communities)}")for i, comm in enumerate(communities[:5]):    print(f"   Community {i+1}: {len(comm)} members")# Assign community labelscommunity_map = {}for i, comm in enumerate(communities):    for node in comm:        community_map[node] = inx.set_node_attributes(G, community_map, 'community')

## 3️⃣ Centrality Analysis\n\nIdentify key players in the network using multiple centrality metrics.

In [ ]:
# Calculate centrality metricsdegree_cent = nx.degree_centrality(G)betweenness_cent = nx.betweenness_centrality(G)closeness_cent = nx.closeness_centrality(G)pagerank = nx.pagerank(G)# Combine into DataFramecentrality_df = pd.DataFrame({    'degree': degree_cent,    'betweenness': betweenness_cent,    'closeness': closeness_cent,    'pagerank': pagerank})# Calculate risk scorecentrality_df['risk_score'] = (    centrality_df['degree'] * 0.3 +    centrality_df['betweenness'] * 0.4 +    centrality_df['pagerank'] * 0.3)print("🎯 Top 10 High-Risk Nodes (by risk score):")print(centrality_df.nlargest(10, 'risk_score'))

## 4️⃣ Suspicious Pattern Detection\n\nIdentify specific AML patterns: rapid movement, circular flows, layering.

In [ ]:
# Detect circular flows (potential layering)cycles = list(nx.simple_cycles(G))print(f"🔄 Circular flows detected: {len(cycles)}")# Detect high-velocity nodeshigh_velocity = centrality_df[centrality_df['degree'] > centrality_df['degree'].quantile(0.90)]print(f"⚡ High-velocity nodes: {len(high_velocity)}")# Detect hub-and-spoke patternsout_degree = dict(G.out_degree())in_degree = dict(G.in_degree())hubs = [node for node, deg in out_degree.items() if deg > 10]print(f"🎯 Hub nodes detected: {len(hubs)}")print("\n⚠️ SUSPICIOUS PATTERNS:")for hub in hubs[:5]:    print(f"   {hub}: {out_degree[hub]} outgoing, {in_degree[hub]} incoming")

## 5️⃣ Network Visualization

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(18, 8))# Layoutpos = nx.spring_layout(G, k=0.5, iterations=50, seed=42)# 1. Community visualizationnode_colors = [community_map[node] for node in G.nodes()]nx.draw_networkx(G, pos, node_color=node_colors, cmap='Set3',                  node_size=100, with_labels=False,                  edge_color='gray', alpha=0.6, ax=axes[0])axes[0].set_title('👥 Network Communities', fontsize=14, fontweight='bold')axes[0].axis('off')# 2. Risk score visualizationnode_sizes = [centrality_df.loc[node, 'risk_score'] * 1000 for node in G.nodes()]node_colors_risk = [centrality_df.loc[node, 'risk_score'] for node in G.nodes()]nx.draw_networkx(G, pos, node_color=node_colors_risk, cmap='YlOrRd',                 node_size=node_sizes, with_labels=False,                 edge_color='gray', alpha=0.6, ax=axes[1])axes[1].set_title('🚨 Risk Score (size & color)', fontsize=14, fontweight='bold')axes[1].axis('off')plt.tight_layout()plt.show()print("\n✅ Network analysis complete!")